In [44]:
import pandas as pd
import re
import os
from tqdm import tqdm

In [45]:
input_folder = "data-temperatures"
output_folder = "output-temperatures"

In [46]:
def process_dutgraph_log(input_file_name):
  """
  Processes a dutgraph.log file and extracts relevant information.

  Args:
    file_path: Path to the dutgraph.log file.

  Returns:
    A pandas DataFrame containing the extracted data.
  """
  input_file_path = os.path.join(input_folder, input_file_name)
  with open(input_file_path, 'r') as f:
    lines = f.readlines()

  # Extract Zone and Slot information
  zone_slot_match = re.search(r'Zone (\d+), Slot (\d+)', ''.join(lines))
  zone = int(zone_slot_match.group(1))
  slot = int(zone_slot_match.group(2))

  # Extract product name
  product_match = re.search(r'Mapping source: .+\\(.*?)\\', ''.join(lines))
  product = product_match.group(1)

  sockets_count = re.search(r'(\d+) sockets', ''.join(lines))
  if sockets_count:
    num_sockets = int(sockets_count.group(1))

#ignore the files with 18 sockets
  info_rows_count = 6
  socket_count = num_sockets

  # Read the data into a pandas DataFrame
  df = pd.read_csv(input_file_path, skiprows=info_rows_count + socket_count)

  cols_to_keep = ['COOLDUTY', 'ES0_TEMP', 'HTR_DUTY', 'SNK_TEMP']
  all_cols = [f"{col}.{socket_num}" for socket_num in range(socket_count) for col in cols_to_keep]

  final_df = df[all_cols]

  output_file_name = f"{zone}_{slot}_{product}.csv"
  output_file_path = os.path.join(output_folder, output_file_name)

  final_df.to_csv(output_file_path) # index=False

  return df[all_cols]

In [47]:
files = os.listdir(input_folder)
for file_name in tqdm(files):
    print(file_name)
    process_dutgraph_log(file_name)


  6%|▌         | 3/50 [00:00<00:01, 28.79it/s]

dutgraph04_Z1S00.log
dutgraph05_Z1S01.log
dutgraph09_Z1S01.log
dutgraph10_Z1S02.log
dutgraph12_Z1S00.log
dutgraph13_Z1S01.log


 10%|█         | 5/50 [00:06<00:58,  1.31s/it]


KeyboardInterrupt: 